In [1]:
import random

import numpy as np
import torch
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from torch import nn
from tqdm import tqdm
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

In [2]:
data = np.load(r'D:\Advancing_ocean_intuition\pacific\last\1day_for_1day_all_variables_intuition_nino_area_surface.npz')
print(
    data.files)  # ['sst_obser', 'slfh', 'sshf', 'ssr', 'str', 'mld', 'analysis_temp', 'u', 'v', 'T_d', 'u_d', 'v_d', 'xx', 'yy', 'dT_dt', 'dT_dx', 'dT_dy']

sst_obser = data['sst_obser'][:, :40, :200]  # (9861, 41, 201)  #
print(sst_obser.shape)  # (9861, 41, 201)
slfh = data['slfh'][:, :40, :200]   # (9861, 41, 201)
sshf = data['sshf'][:, :40, :200]   # (9861, 41, 201)
ssr = data['ssr'][:, :40, :200]   # (9861, 41, 201)   # 可惜这个是 0
str = data['str'][:, :40, :200]   # (9861, 41, 201)
mld = data['mld'][:, :40, :200]   # (9861, 41, 201)
analysis_temp = data['analysis_temp'][:, :40, :200]  # (9861, 41, 201)
u = data['u'][:, :40, :200]  # (9861, 41, 201)
v = data['v'][:, :40, :200]  # (9861, 41, 201)
T_d = data['T_d'][:, :40, :200]  # (9861, 41, 201)
u_d = data['u_d'][:, :40, :200]  # (9861, 41, 201)
v_d = data['v_d'][:, :40, :200]  # (9861, 41, 201)
xx = data['xx'][:, :40, :200]  # (9861, 41, 201)
yy = data['yy'][:, :40, :200]  # (9861, 41, 201)
print(np.min(T_d))
print(np.max(T_d))
# print(xx)  # -5 -4.75 ... 4.75
# print(yy)  # 190  190.25 ... 239.75

dT_dt = data['dT_dt'][:, :40, :200]
dT_dx = data['dT_dx'][:, :40, :200]
dT_dy = data['dT_dy'][:, :40, :200]

['sst_obser', 'slfh', 'sshf', 'ssr', 'str', 'mld', 'analysis_temp', 'u', 'v', 'T_d', 'u_d', 'v_d', 'xx', 'yy', 'dT_dt', 'dT_dx', 'dT_dy']
['sst_obser', 'slfh', 'sshf', 'ssr', 'str', 'mld', 'analysis_temp', 'u', 'v', 'T_d', 'u_d', 'v_d', 'xx', 'yy', 'dT_dt', 'dT_dx', 'dT_dy']
(9861, 40, 200)
(9861, 40, 200)
19.893036
31.218864
19.893036
31.218864


In [3]:
Q_net = (slfh + sshf + ssr + str)
Q_net.shape
Q_net = Q_net / (1025 * 4000 * mld)

In [4]:
ZAdv = u * dT_dx
ZAdv.shape

(9861, 40, 200)

(9861, 40, 200)

In [5]:
MAdv = v * dT_dy
MAdv.shape

(9861, 40, 200)

(9861, 40, 200)

In [6]:
VAdv = v_d * ((sst_obser -T_d ) / mld)
VAdv.shape

(9861, 40, 200)

(9861, 40, 200)

In [7]:
R_origin = dT_dt - (Q_net[:9860,:,:] + ZAdv[:9860,:,:] + MAdv[:9860,:,:] + VAdv[:9860,:,:])
R_origin.shape

(9860, 40, 200)

(9860, 40, 200)

In [10]:
"""我们方法修正项"""

'我们方法修正项'

'我们方法修正项'

In [13]:
data1 = np.load(r'D:\heat_wave\pacific\last\Dt_with_R_e1.npz')
data2 = np.load(r'D:\heat_wave\pacific\last\Dt_with_R_e2.npz')
data3 = np.load(r'D:\heat_wave\pacific\last\Dt_with_R_e3.npz')
data4 = np.load(r'D:\heat_wave\pacific\last\Dt_with_R_e4.npz')
data5 = np.load(r'D:\heat_wave\pacific\last\Dt_with_R_e5.npz')
data1.files

['dt_R']

['dt_R']

In [14]:
dt_R1 = data1['dt_R'][:].reshape(9860,40,200)
dt_R2 = data2['dt_R'][:].reshape(9860,40,200)
dt_R3 = data3['dt_R'][:].reshape(9860,40,200)
dt_R4 = data4['dt_R'][:].reshape(9860,40,200)
dt_R5 = data5['dt_R'][:].reshape(9860,40,200)
dt_R1.shape

(9860, 40, 200)

(9860, 40, 200)

In [15]:
"修正后的R"
R_recur1 = dt_R1 - dT_dt
R_recur2 = dt_R2 - dT_dt
R_recur3 = dt_R3 - dT_dt
R_recur4 = dt_R4 - dT_dt
R_recur5 = dt_R5 - dT_dt
R_recur1

array([[[-0.28933302, -0.05333645, -0.01332911, ...,  0.4865895 ,
          0.41115975,  0.4465634 ],
        [-0.38266695, -0.0688787 , -0.02874806, ...,  0.39565662,
          0.3509448 ,  0.33698338],
        [-0.4408059 , -0.06624164, -0.01722556, ...,  0.32649016,
          0.28285578,  0.3038839 ],
        ...,
        [-0.5242568 , -0.33345646, -0.12883733, ...,  0.03674373,
         -0.06265006,  0.10872991],
        [-0.525598  , -0.39512724, -0.21158215, ...,  0.00786022,
         -0.05260696,  0.05185654],
        [-0.4324652 , -0.32346192, -0.21872598, ...,  0.02395673,
         -0.06527562,  0.01934293]],

       [[-0.1684628 , -0.01135238,  0.01223398, ...,  0.04010908,
         -0.0401582 , -0.07452881],
        [ 0.09938391,  0.36688524,  0.30626085, ..., -0.01066904,
         -0.09954942, -0.07292452],
        [ 0.37076953,  0.6388966 ,  0.54602593, ..., -0.08630226,
         -0.10910824, -0.09242129],
        ...,
        [ 0.09033634,  0.15601511,  0.15940917, ..., -

array([[[-0.28933302, -0.05333645, -0.01332911, ...,  0.4865895 ,
          0.41115975,  0.4465634 ],
        [-0.38266695, -0.0688787 , -0.02874806, ...,  0.39565662,
          0.3509448 ,  0.33698338],
        [-0.4408059 , -0.06624164, -0.01722556, ...,  0.32649016,
          0.28285578,  0.3038839 ],
        ...,
        [-0.5242568 , -0.33345646, -0.12883733, ...,  0.03674373,
         -0.06265006,  0.10872991],
        [-0.525598  , -0.39512724, -0.21158215, ...,  0.00786022,
         -0.05260696,  0.05185654],
        [-0.4324652 , -0.32346192, -0.21872598, ...,  0.02395673,
         -0.06527562,  0.01934293]],

       [[-0.1684628 , -0.01135238,  0.01223398, ...,  0.04010908,
         -0.0401582 , -0.07452881],
        [ 0.09938391,  0.36688524,  0.30626085, ..., -0.01066904,
         -0.09954942, -0.07292452],
        [ 0.37076953,  0.6388966 ,  0.54602593, ..., -0.08630226,
         -0.10910824, -0.09242129],
        ...,
        [ 0.09033634,  0.15601511,  0.15940917, ..., -

In [19]:
R_recur = ((R_recur1 + R_recur2 + R_recur3 + R_recur4 + R_recur5) / 5)
R_recur

array([[[-3.12543452e-01, -5.95308542e-02, -1.75928529e-02, ...,
          4.81393039e-01,  4.07376766e-01,  4.52931315e-01],
        [-4.08123344e-01, -7.70716816e-02, -3.38339880e-02, ...,
          3.90346110e-01,  3.47013056e-01,  3.43133390e-01],
        [-4.68668163e-01, -7.74228945e-02, -2.32474599e-02, ...,
          3.21300149e-01,  2.79046118e-01,  3.09978843e-01],
        ...,
        [-5.49191594e-01, -3.40549827e-01, -1.32836536e-01, ...,
          3.06254569e-02, -7.08688721e-02,  1.12348534e-01],
        [-5.52369118e-01, -4.03622299e-01, -2.16258764e-01, ...,
          1.13031117e-03, -5.98799102e-02,  5.62734380e-02],
        [-4.57709372e-01, -3.30214649e-01, -2.23426849e-01, ...,
          1.73730534e-02, -7.21838847e-02,  2.38319058e-02]],

       [[-1.89451426e-01, -1.61855984e-02,  5.84824150e-03, ...,
          3.47832814e-02, -4.43773344e-02, -6.86048418e-02],
        [ 7.80561939e-02,  3.61433327e-01,  2.99638182e-01, ...,
         -1.62344072e-02, -1.03603818e

array([[[-3.12543452e-01, -5.95308542e-02, -1.75928529e-02, ...,
          4.81393039e-01,  4.07376766e-01,  4.52931315e-01],
        [-4.08123344e-01, -7.70716816e-02, -3.38339880e-02, ...,
          3.90346110e-01,  3.47013056e-01,  3.43133390e-01],
        [-4.68668163e-01, -7.74228945e-02, -2.32474599e-02, ...,
          3.21300149e-01,  2.79046118e-01,  3.09978843e-01],
        ...,
        [-5.49191594e-01, -3.40549827e-01, -1.32836536e-01, ...,
          3.06254569e-02, -7.08688721e-02,  1.12348534e-01],
        [-5.52369118e-01, -4.03622299e-01, -2.16258764e-01, ...,
          1.13031117e-03, -5.98799102e-02,  5.62734380e-02],
        [-4.57709372e-01, -3.30214649e-01, -2.23426849e-01, ...,
          1.73730534e-02, -7.21838847e-02,  2.38319058e-02]],

       [[-1.89451426e-01, -1.61855984e-02,  5.84824150e-03, ...,
          3.47832814e-02, -4.43773344e-02, -6.86048418e-02],
        [ 7.80561939e-02,  3.61433327e-01,  2.99638182e-01, ...,
         -1.62344072e-02, -1.03603818e

In [17]:
# 首先 我们先画 SON季节的贡献图   JJA： 9  10  11 月  即  09/01 - 11/31  当然，会做一个区域平均
# 同时，我们也会对每一年的JJA做一个平均  ---》1993 - 2019年

In [20]:
#  计算索引
import pandas as pd
import datetime

#创建时间范围
start_date = datetime.datetime(1993,1,1)
end_date = datetime.datetime(2019,12,31)
dates = pd.date_range(start = start_date, end = end_date, freq = 'D')

index = dates.get_loc(datetime.datetime(2019,11,30)) 
index

9829

9829

In [21]:
# SON   93
Q_net_SON_93 = Q_net[243:334,:,:]
Q_net_SON_93.shape

ZAdv_SON_93 = ZAdv[243:334,:,:]
MAdv_SON_93 = MAdv[243:334,:,:]
VAdv_SON_93 = VAdv[243:334,:,:]
R_origin_SON_93 = R_origin[243:334,:,:]
dT_dt_SON_93 = dT_dt[243:334,:,:]
R_recur_SON_93 = R_recur[243:334,:,:]

In [22]:
# SON   94
Q_net_SON_94 = Q_net[608:699,:,:]
Q_net_SON_94.shape

ZAdv_SON_94 = ZAdv[608:699,:,:]
MAdv_SON_94 = MAdv[608:699,:,:]
VAdv_SON_94 = VAdv[608:699,:,:]
R_origin_SON_94 = R_origin[608:699,:,:]
dT_dt_SON_94 = dT_dt[608:699,:,:]
R_recur_SON_94 = R_recur[608:699,:,:]

In [23]:
# SON   95
Q_net_SON_95 = Q_net[973:1064,:,:]
Q_net_SON_95.shape

ZAdv_SON_95 = ZAdv[973:1064,:,:]
MAdv_SON_95 = MAdv[973:1064,:,:]
VAdv_SON_95 = VAdv[973:1064,:,:]
R_origin_SON_95 = R_origin[973:1064,:,:]
dT_dt_SON_95 = dT_dt[973:1064,:,:]
R_recur_SON_95 = R_recur[973:1064,:,:]

In [24]:
# SON   96
Q_net_SON_96 = Q_net[1339:1430,:,:]
ZAdv_SON_96 = ZAdv[1339:1430,:,:]
MAdv_SON_96 = MAdv[1339:1430,:,:]
VAdv_SON_96 = VAdv[1339:1430,:,:]
R_origin_SON_96 = R_origin[1339:1430,:,:]
dT_dt_SON_96 = dT_dt[1339:1430,:,:]
R_recur_SON_96 = R_recur[1339:1430,:,:]

In [25]:
# SON   97
Q_net_SON_97 = Q_net[1704:1795,:,:]
ZAdv_SON_97 = ZAdv[1704:1795,:,:]
MAdv_SON_97 = MAdv[1704:1795,:,:]
VAdv_SON_97 = VAdv[1704:1795,:,:]
R_origin_SON_97 = R_origin[1704:1795,:,:]
dT_dt_SON_97 = dT_dt[1704:1795,:,:]
R_recur_SON_97 = R_recur[1704:1795,:,:]

In [26]:
# SON   98
Q_net_SON_98 = Q_net[2069:2160,:,:]
ZAdv_SON_98 = ZAdv[2069:2160,:,:]
MAdv_SON_98 = MAdv[2069:2160,:,:]
VAdv_SON_98 = VAdv[2069:2160,:,:]
R_origin_SON_98 = R_origin[2069:2160,:,:]
dT_dt_SON_98 = dT_dt[2069:2160,:,:]
R_recur_SON_98 = R_recur[2069:2160,:,:]

In [27]:
# SON   99
Q_net_SON_99 = Q_net[2434:2525,:,:]
ZAdv_SON_99 = ZAdv[2434:2525,:,:]
MAdv_SON_99 = MAdv[2434:2525,:,:]
VAdv_SON_99 = VAdv[2434:2525,:,:]
R_origin_SON_99 = R_origin[2434:2525,:,:]
dT_dt_SON_99 = dT_dt[2434:2525,:,:]
R_recur_SON_99 = R_recur[2434:2525,:,:]

In [28]:
# SON   00
Q_net_SON_00 = Q_net[2800:2891,:,:]
ZAdv_SON_00 = ZAdv[2800:2891,:,:]
MAdv_SON_00 = MAdv[2800:2891,:,:]
VAdv_SON_00 = VAdv[2800:2891,:,:]
R_origin_SON_00 = R_origin[2800:2891,:,:]
dT_dt_SON_00 = dT_dt[2800:2891,:,:]
R_recur_SON_00 = R_recur[2800:2891,:,:]

In [29]:
# SON   01
Q_net_SON_01 = Q_net[3165:3256,:,:]
ZAdv_SON_01 = ZAdv[3165:3256,:,:]
MAdv_SON_01 = MAdv[3165:3256,:,:]
VAdv_SON_01 = VAdv[3165:3256,:,:]
R_origin_SON_01 = R_origin[3165:3256,:,:]
dT_dt_SON_01 = dT_dt[3165:3256,:,:]
R_recur_SON_01 = R_recur[3165:3256,:,:]

In [30]:
# SON   02
Q_net_SON_02 = Q_net[3529:3621,:,:]
ZAdv_SON_02 = ZAdv[3529:3621,:,:]
MAdv_SON_02 = MAdv[3529:3621,:,:]
VAdv_SON_02 = VAdv[3529:3621,:,:]
R_origin_SON_02 = R_origin[3529:3621,:,:]
dT_dt_SON_02 = dT_dt[3529:3621,:,:]
R_recur_SON_02 = R_recur[3529:3621,:,:]

In [31]:
# SON   03
Q_net_SON_03 = Q_net[3895:3986,:,:]
ZAdv_SON_03 = ZAdv[3895:3986,:,:]
MAdv_SON_03 = MAdv[3895:3986,:,:]
VAdv_SON_03 = VAdv[3895:3986,:,:]
R_origin_SON_03 = R_origin[3895:3986,:,:]
dT_dt_SON_03 = dT_dt[3895:3986,:,:]
R_recur_SON_03 = R_recur[3895:3986,:,:]

In [32]:
# SON   04
Q_net_SON_04 = Q_net[4261:4352,:,:]
ZAdv_SON_04 = ZAdv[4261:4352,:,:]
MAdv_SON_04 = MAdv[4261:4352,:,:]
VAdv_SON_04 = VAdv[4261:4352,:,:]
R_origin_SON_04 = R_origin[4261:4352,:,:]
dT_dt_SON_04 = dT_dt[4261:4352,:,:]
R_recur_SON_04 = R_recur[4261:4352,:,:]

In [33]:
# SON   05
Q_net_SON_05 = Q_net[4626:4717,:,:]
ZAdv_SON_05 = ZAdv[4626:4717,:,:]
MAdv_SON_05 = MAdv[4626:4717,:,:]
VAdv_SON_05 = VAdv[4626:4717,:,:]
R_origin_SON_05 = R_origin[4626:4717,:,:]
dT_dt_SON_05 = dT_dt[4626:4717,:,:]
R_recur_SON_05 = R_recur[4626:4717,:,:]

In [34]:
# SON   06
Q_net_SON_06 = Q_net[4990:5082,:,:]
ZAdv_SON_06 = ZAdv[4990:5082,:,:]
MAdv_SON_06 = MAdv[4990:5082,:,:]
VAdv_SON_06 = VAdv[4990:5082,:,:]
R_origin_SON_06 = R_origin[4990:5082,:,:]
dT_dt_SON_06 = dT_dt[4990:5082,:,:]
R_recur_SON_06 = R_recur[4990:5082,:,:]

In [35]:
# SON   07
Q_net_SON_07 = Q_net[5356:5447,:,:]
ZAdv_SON_07 = ZAdv[5356:5447,:,:]
MAdv_SON_07 = MAdv[5356:5447,:,:]
VAdv_SON_07 = VAdv[5356:5447,:,:]
R_origin_SON_07 = R_origin[5356:5447,:,:]
dT_dt_SON_07 = dT_dt[5356:5447,:,:]
R_recur_SON_07 = R_recur[5356:5447,:,:]

In [36]:
# SON   08
Q_net_SON_08 = Q_net[5722:5813,:,:]
ZAdv_SON_08 = ZAdv[5722:5813,:,:]
MAdv_SON_08 = MAdv[5722:5813,:,:]
VAdv_SON_08 = VAdv[5722:5813,:,:]
R_origin_SON_08 = R_origin[5722:5813,:,:]
dT_dt_SON_08 = dT_dt[5722:5813,:,:]
R_recur_SON_08 = R_recur[5722:5813,:,:]

In [37]:
# SON   09
Q_net_SON_09 = Q_net[6087:6178,:,:]
ZAdv_SON_09 = ZAdv[6087:6178,:,:]
MAdv_SON_09 = MAdv[6087:6178,:,:]
VAdv_SON_09 = VAdv[6087:6178,:,:]
R_origin_SON_09 = R_origin[6087:6178,:,:]
dT_dt_SON_09 = dT_dt[6087:6178,:,:]
R_recur_SON_09 = R_recur[6087:6178,:,:]

In [38]:
# SON   10
Q_net_SON_10 = Q_net[6452:6543,:,:]
ZAdv_SON_10 = ZAdv[6452:6543,:,:]
MAdv_SON_10 = MAdv[6452:6543,:,:]
VAdv_SON_10 = VAdv[6452:6543,:,:]
R_origin_SON_10 = R_origin[6452:6543,:,:]
dT_dt_SON_10 = dT_dt[6452:6543,:,:]
R_recur_SON_10 = R_recur[6452:6543,:,:]

In [39]:
# SON   11
Q_net_SON_11 = Q_net[6817:6908,:,:]
ZAdv_SON_11 = ZAdv[6817:6908,:,:]
MAdv_SON_11 = MAdv[6817:6908,:,:]
VAdv_SON_11 = VAdv[6817:6908,:,:]
R_origin_SON_11 = R_origin[6817:6908,:,:]
dT_dt_SON_11 = dT_dt[6817:6908,:,:]
R_recur_SON_11 = R_recur[6817:6908,:,:]

In [40]:
# SON   12
Q_net_SON_12 = Q_net[7183:7274,:,:]
ZAdv_SON_12 = ZAdv[7183:7274,:,:]
MAdv_SON_12 = MAdv[7183:7274,:,:]
VAdv_SON_12 = VAdv[7183:7274,:,:]
R_origin_SON_12 = R_origin[7183:7274,:,:]
dT_dt_SON_12 = dT_dt[7183:7274,:,:]
R_recur_SON_12 = R_recur[7183:7274,:,:]

In [41]:
# SON   13
Q_net_SON_13 = Q_net[7548:7639,:,:]
ZAdv_SON_13 = ZAdv[7548:7639,:,:]
MAdv_SON_13 = MAdv[7548:7639,:,:]
VAdv_SON_13 = VAdv[7548:7639,:,:]
R_origin_SON_13 = R_origin[7548:7639,:,:]
dT_dt_SON_13 = dT_dt[7548:7639,:,:]
R_recur_SON_13 = R_recur[7548:7639,:,:]

In [42]:
# SON   14
Q_net_SON_14 = Q_net[7913:8004,:,:]
ZAdv_SON_14 = ZAdv[7913:8004,:,:]
MAdv_SON_14 = MAdv[7913:8004,:,:]
VAdv_SON_14 = VAdv[7913:8004,:,:]
R_origin_SON_14 = R_origin[7913:8004,:,:]
dT_dt_SON_14 = dT_dt[7913:8004,:,:]
R_recur_SON_14 = R_recur[7913:8004,:,:]

In [43]:
# SON   15
Q_net_SON_15 = Q_net[8278:8369,:,:]
ZAdv_SON_15 = ZAdv[8278:8369,:,:]
MAdv_SON_15 = MAdv[8278:8369,:,:]
VAdv_SON_15 = VAdv[8278:8369,:,:]
R_origin_SON_15 = R_origin[8278:8369,:,:]
dT_dt_SON_15 = dT_dt[8278:8369,:,:]
R_recur_SON_15 = R_recur[8278:8369,:,:]

In [44]:
# SON   16
Q_net_SON_16 = Q_net[8644:8735,:,:]
ZAdv_SON_16 = ZAdv[8644:8735,:,:]
MAdv_SON_16 = MAdv[8644:8735,:,:]
VAdv_SON_16 = VAdv[8644:8735,:,:]
R_origin_SON_16 = R_origin[8644:8735,:,:]
dT_dt_SON_16 = dT_dt[8644:8735,:,:]
R_recur_SON_16 = R_recur[8644:8735,:,:]

In [45]:
# SON   17
Q_net_SON_17 = Q_net[9009:9100,:,:]
ZAdv_SON_17 = ZAdv[9009:9100,:,:]
MAdv_SON_17 = MAdv[9009:9100,:,:]
VAdv_SON_17 = VAdv[9009:9100,:,:]
R_origin_SON_17 = R_origin[9009:9100,:,:]
dT_dt_SON_17 = dT_dt[9009:9100,:,:]
R_recur_SON_17 = R_recur[9009:9100,:,:]

In [46]:
# SON   18
Q_net_SON_18 = Q_net[9374:9465,:,:]
ZAdv_SON_18 = ZAdv[9374:9465,:,:]
MAdv_SON_18 = MAdv[9374:9465,:,:]
VAdv_SON_18 = VAdv[9374:9465,:,:]
R_origin_SON_18 = R_origin[9374:9465,:,:]
dT_dt_SON_18 = dT_dt[9374:9465,:,:]
R_recur_SON_18 = R_recur[9374:9465,:,:]

In [47]:
# SON   19
Q_net_SON_19 = Q_net[9739:9830,:,:]
ZAdv_SON_19 = ZAdv[9739:9830,:,:]
MAdv_SON_19 = MAdv[9739:9830,:,:]
VAdv_SON_19 = VAdv[9739:9830,:,:]
R_origin_SON_19 = R_origin[9739:9830,:,:]
dT_dt_SON_19 = dT_dt[9739:9830,:,:]
R_recur_SON_19 = R_recur[9739:9830,:,:]

In [48]:
Q_net_SON_93 = np.mean(Q_net_SON_93, axis = (1,2)).reshape(-1,1)
Q_net_SON_94 = np.mean(Q_net_SON_94, axis = (1,2)).reshape(-1,1)
Q_net_SON_95 = np.mean(Q_net_SON_95, axis = (1,2)).reshape(-1,1)
Q_net_SON_96 = np.mean(Q_net_SON_96, axis = (1,2)).reshape(-1,1)
Q_net_SON_97 = np.mean(Q_net_SON_97, axis = (1,2)).reshape(-1,1)
Q_net_SON_98 = np.mean(Q_net_SON_98, axis = (1,2)).reshape(-1,1)
Q_net_SON_99 = np.mean(Q_net_SON_99, axis = (1,2)).reshape(-1,1)
Q_net_SON_00 = np.mean(Q_net_SON_00, axis = (1,2)).reshape(-1,1)
Q_net_SON_01 = np.mean(Q_net_SON_01, axis = (1,2)).reshape(-1,1)
Q_net_SON_02 = np.mean(Q_net_SON_02, axis = (1,2)).reshape(-1,1)
Q_net_SON_03 = np.mean(Q_net_SON_03, axis = (1,2)).reshape(-1,1)
Q_net_SON_04 = np.mean(Q_net_SON_04, axis = (1,2)).reshape(-1,1)
Q_net_SON_05 = np.mean(Q_net_SON_05, axis = (1,2)).reshape(-1,1)
Q_net_SON_06 = np.mean(Q_net_SON_06, axis = (1,2)).reshape(-1,1)
Q_net_SON_07 = np.mean(Q_net_SON_07, axis = (1,2)).reshape(-1,1)
Q_net_SON_08 = np.mean(Q_net_SON_08, axis = (1,2)).reshape(-1,1)
Q_net_SON_09 = np.mean(Q_net_SON_09, axis = (1,2)).reshape(-1,1)
Q_net_SON_10 = np.mean(Q_net_SON_10, axis = (1,2)).reshape(-1,1)
Q_net_SON_11 = np.mean(Q_net_SON_11, axis = (1,2)).reshape(-1,1)
Q_net_SON_12 = np.mean(Q_net_SON_12, axis = (1,2)).reshape(-1,1)
Q_net_SON_13 = np.mean(Q_net_SON_13, axis = (1,2)).reshape(-1,1)
Q_net_SON_14 = np.mean(Q_net_SON_14, axis = (1,2)).reshape(-1,1)
Q_net_SON_15 = np.mean(Q_net_SON_15, axis = (1,2)).reshape(-1,1)
Q_net_SON_16 = np.mean(Q_net_SON_16, axis = (1,2)).reshape(-1,1)
Q_net_SON_17 = np.mean(Q_net_SON_17, axis = (1,2)).reshape(-1,1)
Q_net_SON_18 = np.mean(Q_net_SON_18, axis = (1,2)).reshape(-1,1)
Q_net_SON_19 = np.mean(Q_net_SON_19, axis = (1,2)).reshape(-1,1)

In [49]:
Q_net_SON_93 = np.mean(Q_net_SON_93).reshape(-1,1)
Q_net_SON_94 = np.mean(Q_net_SON_94).reshape(-1,1)
Q_net_SON_95 = np.mean(Q_net_SON_95).reshape(-1,1)
Q_net_SON_96 = np.mean(Q_net_SON_96).reshape(-1,1)
Q_net_SON_97 = np.mean(Q_net_SON_97).reshape(-1,1)
Q_net_SON_98 = np.mean(Q_net_SON_98).reshape(-1,1)
Q_net_SON_99 = np.mean(Q_net_SON_99).reshape(-1,1)
Q_net_SON_00 = np.mean(Q_net_SON_00).reshape(-1,1)
Q_net_SON_01 = np.mean(Q_net_SON_01).reshape(-1,1)
Q_net_SON_02 = np.mean(Q_net_SON_02).reshape(-1,1)
Q_net_SON_03 = np.mean(Q_net_SON_03).reshape(-1,1)
Q_net_SON_04 = np.mean(Q_net_SON_04).reshape(-1,1)
Q_net_SON_05 = np.mean(Q_net_SON_05).reshape(-1,1)
Q_net_SON_06 = np.mean(Q_net_SON_06).reshape(-1,1)
Q_net_SON_07 = np.mean(Q_net_SON_07).reshape(-1,1)
Q_net_SON_08 = np.mean(Q_net_SON_08).reshape(-1,1)
Q_net_SON_09 = np.mean(Q_net_SON_09).reshape(-1,1)
Q_net_SON_10 = np.mean(Q_net_SON_10).reshape(-1,1)
Q_net_SON_11 = np.mean(Q_net_SON_11).reshape(-1,1)
Q_net_SON_12 = np.mean(Q_net_SON_12).reshape(-1,1)
Q_net_SON_13 = np.mean(Q_net_SON_13).reshape(-1,1)
Q_net_SON_14 = np.mean(Q_net_SON_14).reshape(-1,1)
Q_net_SON_15 = np.mean(Q_net_SON_15).reshape(-1,1)
Q_net_SON_16 = np.mean(Q_net_SON_16).reshape(-1,1)
Q_net_SON_17 = np.mean(Q_net_SON_17).reshape(-1,1)
Q_net_SON_18 = np.mean(Q_net_SON_18).reshape(-1,1)
Q_net_SON_19 = np.mean(Q_net_SON_19).reshape(-1,1)

In [50]:
ZAdv_SON_93 = np.mean(ZAdv_SON_93, axis = (1,2)).reshape(-1,1)
ZAdv_SON_94 = np.mean(ZAdv_SON_94, axis = (1,2)).reshape(-1,1)
ZAdv_SON_95 = np.mean(ZAdv_SON_95, axis = (1,2)).reshape(-1,1)
ZAdv_SON_96 = np.mean(ZAdv_SON_96, axis = (1,2)).reshape(-1,1)
ZAdv_SON_97 = np.mean(ZAdv_SON_97, axis = (1,2)).reshape(-1,1)
ZAdv_SON_98 = np.mean(ZAdv_SON_98, axis = (1,2)).reshape(-1,1)
ZAdv_SON_99 = np.mean(ZAdv_SON_99, axis = (1,2)).reshape(-1,1)
ZAdv_SON_00 = np.mean(ZAdv_SON_00, axis = (1,2)).reshape(-1,1)
ZAdv_SON_01 = np.mean(ZAdv_SON_01, axis = (1,2)).reshape(-1,1)
ZAdv_SON_02 = np.mean(ZAdv_SON_02, axis = (1,2)).reshape(-1,1)
ZAdv_SON_03 = np.mean(ZAdv_SON_03, axis = (1,2)).reshape(-1,1)
ZAdv_SON_04 = np.mean(ZAdv_SON_04, axis = (1,2)).reshape(-1,1)
ZAdv_SON_05 = np.mean(ZAdv_SON_05, axis = (1,2)).reshape(-1,1)
ZAdv_SON_06 = np.mean(ZAdv_SON_06, axis = (1,2)).reshape(-1,1)
ZAdv_SON_07 = np.mean(ZAdv_SON_07, axis = (1,2)).reshape(-1,1)
ZAdv_SON_08 = np.mean(ZAdv_SON_08, axis = (1,2)).reshape(-1,1)
ZAdv_SON_09 = np.mean(ZAdv_SON_09, axis = (1,2)).reshape(-1,1)
ZAdv_SON_10 = np.mean(ZAdv_SON_10, axis = (1,2)).reshape(-1,1)
ZAdv_SON_11 = np.mean(ZAdv_SON_11, axis = (1,2)).reshape(-1,1)
ZAdv_SON_12 = np.mean(ZAdv_SON_12, axis = (1,2)).reshape(-1,1)
ZAdv_SON_13 = np.mean(ZAdv_SON_13, axis = (1,2)).reshape(-1,1)
ZAdv_SON_14 = np.mean(ZAdv_SON_14, axis = (1,2)).reshape(-1,1)
ZAdv_SON_15 = np.mean(ZAdv_SON_15, axis = (1,2)).reshape(-1,1)
ZAdv_SON_16 = np.mean(ZAdv_SON_16, axis = (1,2)).reshape(-1,1)
ZAdv_SON_17 = np.mean(ZAdv_SON_17, axis = (1,2)).reshape(-1,1)
ZAdv_SON_18 = np.mean(ZAdv_SON_18, axis = (1,2)).reshape(-1,1)
ZAdv_SON_19 = np.mean(ZAdv_SON_19, axis = (1,2)).reshape(-1,1)

In [51]:
ZAdv_SON_93 = np.mean(ZAdv_SON_93).reshape(-1,1)
ZAdv_SON_94 = np.mean(ZAdv_SON_94).reshape(-1,1)
ZAdv_SON_95 = np.mean(ZAdv_SON_95).reshape(-1,1)
ZAdv_SON_96 = np.mean(ZAdv_SON_96).reshape(-1,1)
ZAdv_SON_97 = np.mean(ZAdv_SON_97).reshape(-1,1)
ZAdv_SON_98 = np.mean(ZAdv_SON_98).reshape(-1,1)
ZAdv_SON_99 = np.mean(ZAdv_SON_99).reshape(-1,1)
ZAdv_SON_00 = np.mean(ZAdv_SON_00).reshape(-1,1)
ZAdv_SON_01 = np.mean(ZAdv_SON_01).reshape(-1,1)
ZAdv_SON_02 = np.mean(ZAdv_SON_02).reshape(-1,1)
ZAdv_SON_03 = np.mean(ZAdv_SON_03).reshape(-1,1)
ZAdv_SON_04 = np.mean(ZAdv_SON_04).reshape(-1,1)
ZAdv_SON_05 = np.mean(ZAdv_SON_05).reshape(-1,1)
ZAdv_SON_06 = np.mean(ZAdv_SON_06).reshape(-1,1)
ZAdv_SON_07 = np.mean(ZAdv_SON_07).reshape(-1,1)
ZAdv_SON_08 = np.mean(ZAdv_SON_08).reshape(-1,1)
ZAdv_SON_09 = np.mean(ZAdv_SON_09).reshape(-1,1)
ZAdv_SON_10 = np.mean(ZAdv_SON_10).reshape(-1,1)
ZAdv_SON_11 = np.mean(ZAdv_SON_11).reshape(-1,1)
ZAdv_SON_12 = np.mean(ZAdv_SON_12).reshape(-1,1)
ZAdv_SON_13 = np.mean(ZAdv_SON_13).reshape(-1,1)
ZAdv_SON_14 = np.mean(ZAdv_SON_14).reshape(-1,1)
ZAdv_SON_15 = np.mean(ZAdv_SON_15).reshape(-1,1)
ZAdv_SON_16 = np.mean(ZAdv_SON_16).reshape(-1,1)
ZAdv_SON_17 = np.mean(ZAdv_SON_17).reshape(-1,1)
ZAdv_SON_18 = np.mean(ZAdv_SON_18).reshape(-1,1)
ZAdv_SON_19 = np.mean(ZAdv_SON_19).reshape(-1,1)

In [52]:
MAdv_SON_93 = np.mean(MAdv_SON_93, axis = (1,2)).reshape(-1,1)
MAdv_SON_94 = np.mean(MAdv_SON_94, axis = (1,2)).reshape(-1,1)
MAdv_SON_95 = np.mean(MAdv_SON_95, axis = (1,2)).reshape(-1,1)
MAdv_SON_96 = np.mean(MAdv_SON_96, axis = (1,2)).reshape(-1,1)
MAdv_SON_97 = np.mean(MAdv_SON_97, axis = (1,2)).reshape(-1,1)
MAdv_SON_98 = np.mean(MAdv_SON_98, axis = (1,2)).reshape(-1,1)
MAdv_SON_99 = np.mean(MAdv_SON_99, axis = (1,2)).reshape(-1,1)
MAdv_SON_00 = np.mean(MAdv_SON_00, axis = (1,2)).reshape(-1,1)
MAdv_SON_01 = np.mean(MAdv_SON_01, axis = (1,2)).reshape(-1,1)
MAdv_SON_02 = np.mean(MAdv_SON_02, axis = (1,2)).reshape(-1,1)
MAdv_SON_03 = np.mean(MAdv_SON_03, axis = (1,2)).reshape(-1,1)
MAdv_SON_04 = np.mean(MAdv_SON_04, axis = (1,2)).reshape(-1,1)
MAdv_SON_05 = np.mean(MAdv_SON_05, axis = (1,2)).reshape(-1,1)
MAdv_SON_06 = np.mean(MAdv_SON_06, axis = (1,2)).reshape(-1,1)
MAdv_SON_07 = np.mean(MAdv_SON_07, axis = (1,2)).reshape(-1,1)
MAdv_SON_08 = np.mean(MAdv_SON_08, axis = (1,2)).reshape(-1,1)
MAdv_SON_09 = np.mean(MAdv_SON_09, axis = (1,2)).reshape(-1,1)
MAdv_SON_10 = np.mean(MAdv_SON_10, axis = (1,2)).reshape(-1,1)
MAdv_SON_11 = np.mean(MAdv_SON_11, axis = (1,2)).reshape(-1,1)
MAdv_SON_12 = np.mean(MAdv_SON_12, axis = (1,2)).reshape(-1,1)
MAdv_SON_13 = np.mean(MAdv_SON_13, axis = (1,2)).reshape(-1,1)
MAdv_SON_14 = np.mean(MAdv_SON_14, axis = (1,2)).reshape(-1,1)
MAdv_SON_15 = np.mean(MAdv_SON_15, axis = (1,2)).reshape(-1,1)
MAdv_SON_16 = np.mean(MAdv_SON_16, axis = (1,2)).reshape(-1,1)
MAdv_SON_17 = np.mean(MAdv_SON_17, axis = (1,2)).reshape(-1,1)
MAdv_SON_18 = np.mean(MAdv_SON_18, axis = (1,2)).reshape(-1,1)
MAdv_SON_19 = np.mean(MAdv_SON_19, axis = (1,2)).reshape(-1,1)

In [53]:
MAdv_SON_93 = np.mean(MAdv_SON_93).reshape(-1,1)
MAdv_SON_94 = np.mean(MAdv_SON_94).reshape(-1,1)
MAdv_SON_95 = np.mean(MAdv_SON_95).reshape(-1,1)
MAdv_SON_96 = np.mean(MAdv_SON_96).reshape(-1,1)
MAdv_SON_97 = np.mean(MAdv_SON_97).reshape(-1,1)
MAdv_SON_98 = np.mean(MAdv_SON_98).reshape(-1,1)
MAdv_SON_99 = np.mean(MAdv_SON_99).reshape(-1,1)
MAdv_SON_00 = np.mean(MAdv_SON_00).reshape(-1,1)
MAdv_SON_01 = np.mean(MAdv_SON_01).reshape(-1,1)
MAdv_SON_02 = np.mean(MAdv_SON_02).reshape(-1,1)
MAdv_SON_03 = np.mean(MAdv_SON_03).reshape(-1,1)
MAdv_SON_04 = np.mean(MAdv_SON_04).reshape(-1,1)
MAdv_SON_05 = np.mean(MAdv_SON_05).reshape(-1,1)
MAdv_SON_06 = np.mean(MAdv_SON_06).reshape(-1,1)
MAdv_SON_07 = np.mean(MAdv_SON_07).reshape(-1,1)
MAdv_SON_08 = np.mean(MAdv_SON_08).reshape(-1,1)
MAdv_SON_09 = np.mean(MAdv_SON_09).reshape(-1,1)
MAdv_SON_10 = np.mean(MAdv_SON_10).reshape(-1,1)
MAdv_SON_11 = np.mean(MAdv_SON_11).reshape(-1,1)
MAdv_SON_12 = np.mean(MAdv_SON_12).reshape(-1,1)
MAdv_SON_13 = np.mean(MAdv_SON_13).reshape(-1,1)
MAdv_SON_14 = np.mean(MAdv_SON_14).reshape(-1,1)
MAdv_SON_15 = np.mean(MAdv_SON_15).reshape(-1,1)
MAdv_SON_16 = np.mean(MAdv_SON_16).reshape(-1,1)
MAdv_SON_17 = np.mean(MAdv_SON_17).reshape(-1,1)
MAdv_SON_18 = np.mean(MAdv_SON_18).reshape(-1,1)
MAdv_SON_19 = np.mean(MAdv_SON_19).reshape(-1,1)

In [54]:
VAdv_SON_93 = np.mean(VAdv_SON_93, axis = (1,2)).reshape(-1,1)
VAdv_SON_94 = np.mean(VAdv_SON_94, axis = (1,2)).reshape(-1,1)
VAdv_SON_95 = np.mean(VAdv_SON_95, axis = (1,2)).reshape(-1,1)
VAdv_SON_96 = np.mean(VAdv_SON_96, axis = (1,2)).reshape(-1,1)
VAdv_SON_97 = np.mean(VAdv_SON_97, axis = (1,2)).reshape(-1,1)
VAdv_SON_98 = np.mean(VAdv_SON_98, axis = (1,2)).reshape(-1,1)
VAdv_SON_99 = np.mean(VAdv_SON_99, axis = (1,2)).reshape(-1,1)
VAdv_SON_00 = np.mean(VAdv_SON_00, axis = (1,2)).reshape(-1,1)
VAdv_SON_01 = np.mean(VAdv_SON_01, axis = (1,2)).reshape(-1,1)
VAdv_SON_02 = np.mean(VAdv_SON_02, axis = (1,2)).reshape(-1,1)
VAdv_SON_03 = np.mean(VAdv_SON_03, axis = (1,2)).reshape(-1,1)
VAdv_SON_04 = np.mean(VAdv_SON_04, axis = (1,2)).reshape(-1,1)
VAdv_SON_05 = np.mean(VAdv_SON_05, axis = (1,2)).reshape(-1,1)
VAdv_SON_06 = np.mean(VAdv_SON_06, axis = (1,2)).reshape(-1,1)
VAdv_SON_07 = np.mean(VAdv_SON_07, axis = (1,2)).reshape(-1,1)
VAdv_SON_08 = np.mean(VAdv_SON_08, axis = (1,2)).reshape(-1,1)
VAdv_SON_09 = np.mean(VAdv_SON_09, axis = (1,2)).reshape(-1,1)
VAdv_SON_10 = np.mean(VAdv_SON_10, axis = (1,2)).reshape(-1,1)
VAdv_SON_11 = np.mean(VAdv_SON_11, axis = (1,2)).reshape(-1,1)
VAdv_SON_12 = np.mean(VAdv_SON_12, axis = (1,2)).reshape(-1,1)
VAdv_SON_13 = np.mean(VAdv_SON_13, axis = (1,2)).reshape(-1,1)
VAdv_SON_14 = np.mean(VAdv_SON_14, axis = (1,2)).reshape(-1,1)
VAdv_SON_15 = np.mean(VAdv_SON_15, axis = (1,2)).reshape(-1,1)
VAdv_SON_16 = np.mean(VAdv_SON_16, axis = (1,2)).reshape(-1,1)
VAdv_SON_17 = np.mean(VAdv_SON_17, axis = (1,2)).reshape(-1,1)
VAdv_SON_18 = np.mean(VAdv_SON_18, axis = (1,2)).reshape(-1,1)
VAdv_SON_19 = np.mean(VAdv_SON_19, axis = (1,2)).reshape(-1,1)

In [55]:
VAdv_SON_93 = np.mean(VAdv_SON_93).reshape(-1,1)
VAdv_SON_94 = np.mean(VAdv_SON_94).reshape(-1,1)
VAdv_SON_95 = np.mean(VAdv_SON_95).reshape(-1,1)
VAdv_SON_96 = np.mean(VAdv_SON_96).reshape(-1,1)
VAdv_SON_97 = np.mean(VAdv_SON_97).reshape(-1,1)
VAdv_SON_98 = np.mean(VAdv_SON_98).reshape(-1,1)
VAdv_SON_99 = np.mean(VAdv_SON_99).reshape(-1,1)
VAdv_SON_00 = np.mean(VAdv_SON_00).reshape(-1,1)
VAdv_SON_01 = np.mean(VAdv_SON_01).reshape(-1,1)
VAdv_SON_02 = np.mean(VAdv_SON_02).reshape(-1,1)
VAdv_SON_03 = np.mean(VAdv_SON_03).reshape(-1,1)
VAdv_SON_04 = np.mean(VAdv_SON_04).reshape(-1,1)
VAdv_SON_05 = np.mean(VAdv_SON_05).reshape(-1,1)
VAdv_SON_06 = np.mean(VAdv_SON_06).reshape(-1,1)
VAdv_SON_07 = np.mean(VAdv_SON_07).reshape(-1,1)
VAdv_SON_08 = np.mean(VAdv_SON_08).reshape(-1,1)
VAdv_SON_09 = np.mean(VAdv_SON_09).reshape(-1,1)
VAdv_SON_10 = np.mean(VAdv_SON_10).reshape(-1,1)
VAdv_SON_11 = np.mean(VAdv_SON_11).reshape(-1,1)
VAdv_SON_12 = np.mean(VAdv_SON_12).reshape(-1,1)
VAdv_SON_13 = np.mean(VAdv_SON_13).reshape(-1,1)
VAdv_SON_14 = np.mean(VAdv_SON_14).reshape(-1,1)
VAdv_SON_15 = np.mean(VAdv_SON_15).reshape(-1,1)
VAdv_SON_16 = np.mean(VAdv_SON_16).reshape(-1,1)
VAdv_SON_17 = np.mean(VAdv_SON_17).reshape(-1,1)
VAdv_SON_18 = np.mean(VAdv_SON_18).reshape(-1,1)
VAdv_SON_19 = np.mean(VAdv_SON_19).reshape(-1,1)

In [56]:
R_origin_SON_93 = np.mean(R_origin_SON_93, axis = (1,2)).reshape(-1,1)
R_origin_SON_94 = np.mean(R_origin_SON_94, axis = (1,2)).reshape(-1,1)
R_origin_SON_95 = np.mean(R_origin_SON_95, axis = (1,2)).reshape(-1,1)
R_origin_SON_96 = np.mean(R_origin_SON_96, axis = (1,2)).reshape(-1,1)
R_origin_SON_97 = np.mean(R_origin_SON_97, axis = (1,2)).reshape(-1,1)
R_origin_SON_98 = np.mean(R_origin_SON_98, axis = (1,2)).reshape(-1,1)
R_origin_SON_99 = np.mean(R_origin_SON_99, axis = (1,2)).reshape(-1,1)
R_origin_SON_00 = np.mean(R_origin_SON_00, axis = (1,2)).reshape(-1,1)
R_origin_SON_01 = np.mean(R_origin_SON_01, axis = (1,2)).reshape(-1,1)
R_origin_SON_02 = np.mean(R_origin_SON_02, axis = (1,2)).reshape(-1,1)
R_origin_SON_03 = np.mean(R_origin_SON_03, axis = (1,2)).reshape(-1,1)
R_origin_SON_04 = np.mean(R_origin_SON_04, axis = (1,2)).reshape(-1,1)
R_origin_SON_05 = np.mean(R_origin_SON_05, axis = (1,2)).reshape(-1,1)
R_origin_SON_06 = np.mean(R_origin_SON_06, axis = (1,2)).reshape(-1,1)
R_origin_SON_07 = np.mean(R_origin_SON_07, axis = (1,2)).reshape(-1,1)
R_origin_SON_08 = np.mean(R_origin_SON_08, axis = (1,2)).reshape(-1,1)
R_origin_SON_09 = np.mean(R_origin_SON_09, axis = (1,2)).reshape(-1,1)
R_origin_SON_10 = np.mean(R_origin_SON_10, axis = (1,2)).reshape(-1,1)
R_origin_SON_11 = np.mean(R_origin_SON_11, axis = (1,2)).reshape(-1,1)
R_origin_SON_12 = np.mean(R_origin_SON_12, axis = (1,2)).reshape(-1,1)
R_origin_SON_13 = np.mean(R_origin_SON_13, axis = (1,2)).reshape(-1,1)
R_origin_SON_14 = np.mean(R_origin_SON_14, axis = (1,2)).reshape(-1,1)
R_origin_SON_15 = np.mean(R_origin_SON_15, axis = (1,2)).reshape(-1,1)
R_origin_SON_16 = np.mean(R_origin_SON_16, axis = (1,2)).reshape(-1,1)
R_origin_SON_17 = np.mean(R_origin_SON_17, axis = (1,2)).reshape(-1,1)
R_origin_SON_18 = np.mean(R_origin_SON_18, axis = (1,2)).reshape(-1,1)
R_origin_SON_19 = np.mean(R_origin_SON_19, axis = (1,2)).reshape(-1,1)

In [57]:
R_origin_SON_93 = np.mean(R_origin_SON_93).reshape(-1,1)
R_origin_SON_94 = np.mean(R_origin_SON_94).reshape(-1,1)
R_origin_SON_95 = np.mean(R_origin_SON_95).reshape(-1,1)
R_origin_SON_96 = np.mean(R_origin_SON_96).reshape(-1,1)
R_origin_SON_97 = np.mean(R_origin_SON_97).reshape(-1,1)
R_origin_SON_98 = np.mean(R_origin_SON_98).reshape(-1,1)
R_origin_SON_99 = np.mean(R_origin_SON_99).reshape(-1,1)
R_origin_SON_00 = np.mean(R_origin_SON_00).reshape(-1,1)
R_origin_SON_01 = np.mean(R_origin_SON_01).reshape(-1,1)
R_origin_SON_02 = np.mean(R_origin_SON_02).reshape(-1,1)
R_origin_SON_03 = np.mean(R_origin_SON_03).reshape(-1,1)
R_origin_SON_04 = np.mean(R_origin_SON_04).reshape(-1,1)
R_origin_SON_05 = np.mean(R_origin_SON_05).reshape(-1,1)
R_origin_SON_06 = np.mean(R_origin_SON_06).reshape(-1,1)
R_origin_SON_07 = np.mean(R_origin_SON_07).reshape(-1,1)
R_origin_SON_08 = np.mean(R_origin_SON_08).reshape(-1,1)
R_origin_SON_09 = np.mean(R_origin_SON_09).reshape(-1,1)
R_origin_SON_10 = np.mean(R_origin_SON_10).reshape(-1,1)
R_origin_SON_11 = np.mean(R_origin_SON_11).reshape(-1,1)
R_origin_SON_12 = np.mean(R_origin_SON_12).reshape(-1,1)
R_origin_SON_13 = np.mean(R_origin_SON_13).reshape(-1,1)
R_origin_SON_14 = np.mean(R_origin_SON_14).reshape(-1,1)
R_origin_SON_15 = np.mean(R_origin_SON_15).reshape(-1,1)
R_origin_SON_16 = np.mean(R_origin_SON_16).reshape(-1,1)
R_origin_SON_17 = np.mean(R_origin_SON_17).reshape(-1,1)
R_origin_SON_18 = np.mean(R_origin_SON_18).reshape(-1,1)
R_origin_SON_19 = np.mean(R_origin_SON_19).reshape(-1,1)

In [58]:
dT_dt_SON_93 = np.mean(dT_dt_SON_93, axis = (1,2)).reshape(-1,1)
dT_dt_SON_94 = np.mean(dT_dt_SON_94, axis = (1,2)).reshape(-1,1)
dT_dt_SON_95 = np.mean(dT_dt_SON_95, axis = (1,2)).reshape(-1,1)
dT_dt_SON_96 = np.mean(dT_dt_SON_96, axis = (1,2)).reshape(-1,1)
dT_dt_SON_97 = np.mean(dT_dt_SON_97, axis = (1,2)).reshape(-1,1)
dT_dt_SON_98 = np.mean(dT_dt_SON_98, axis = (1,2)).reshape(-1,1)
dT_dt_SON_99 = np.mean(dT_dt_SON_99, axis = (1,2)).reshape(-1,1)
dT_dt_SON_00 = np.mean(dT_dt_SON_00, axis = (1,2)).reshape(-1,1)
dT_dt_SON_01 = np.mean(dT_dt_SON_01, axis = (1,2)).reshape(-1,1)
dT_dt_SON_02 = np.mean(dT_dt_SON_02, axis = (1,2)).reshape(-1,1)
dT_dt_SON_03 = np.mean(dT_dt_SON_03, axis = (1,2)).reshape(-1,1)
dT_dt_SON_04 = np.mean(dT_dt_SON_04, axis = (1,2)).reshape(-1,1)
dT_dt_SON_05 = np.mean(dT_dt_SON_05, axis = (1,2)).reshape(-1,1)
dT_dt_SON_06 = np.mean(dT_dt_SON_06, axis = (1,2)).reshape(-1,1)
dT_dt_SON_07 = np.mean(dT_dt_SON_07, axis = (1,2)).reshape(-1,1)
dT_dt_SON_08 = np.mean(dT_dt_SON_08, axis = (1,2)).reshape(-1,1)
dT_dt_SON_09 = np.mean(dT_dt_SON_09, axis = (1,2)).reshape(-1,1)
dT_dt_SON_10 = np.mean(dT_dt_SON_10, axis = (1,2)).reshape(-1,1)
dT_dt_SON_11 = np.mean(dT_dt_SON_11, axis = (1,2)).reshape(-1,1)
dT_dt_SON_12 = np.mean(dT_dt_SON_12, axis = (1,2)).reshape(-1,1)
dT_dt_SON_13 = np.mean(dT_dt_SON_13, axis = (1,2)).reshape(-1,1)
dT_dt_SON_14 = np.mean(dT_dt_SON_14, axis = (1,2)).reshape(-1,1)
dT_dt_SON_15 = np.mean(dT_dt_SON_15, axis = (1,2)).reshape(-1,1)
dT_dt_SON_16 = np.mean(dT_dt_SON_16, axis = (1,2)).reshape(-1,1)
dT_dt_SON_17 = np.mean(dT_dt_SON_17, axis = (1,2)).reshape(-1,1)
dT_dt_SON_18 = np.mean(dT_dt_SON_18, axis = (1,2)).reshape(-1,1)
dT_dt_SON_19 = np.mean(dT_dt_SON_19, axis = (1,2)).reshape(-1,1)

In [59]:
dT_dt_SON_93 = np.mean(dT_dt_SON_93).reshape(-1,1)
dT_dt_SON_94 = np.mean(dT_dt_SON_94).reshape(-1,1)
dT_dt_SON_95 = np.mean(dT_dt_SON_95).reshape(-1,1)
dT_dt_SON_96 = np.mean(dT_dt_SON_96).reshape(-1,1)
dT_dt_SON_97 = np.mean(dT_dt_SON_97).reshape(-1,1)
dT_dt_SON_98 = np.mean(dT_dt_SON_98).reshape(-1,1)
dT_dt_SON_99 = np.mean(dT_dt_SON_99).reshape(-1,1)
dT_dt_SON_00 = np.mean(dT_dt_SON_00).reshape(-1,1)
dT_dt_SON_01 = np.mean(dT_dt_SON_01).reshape(-1,1)
dT_dt_SON_02 = np.mean(dT_dt_SON_02).reshape(-1,1)
dT_dt_SON_03 = np.mean(dT_dt_SON_03).reshape(-1,1)
dT_dt_SON_04 = np.mean(dT_dt_SON_04).reshape(-1,1)
dT_dt_SON_05 = np.mean(dT_dt_SON_05).reshape(-1,1)
dT_dt_SON_06 = np.mean(dT_dt_SON_06).reshape(-1,1)
dT_dt_SON_07 = np.mean(dT_dt_SON_07).reshape(-1,1)
dT_dt_SON_08 = np.mean(dT_dt_SON_08).reshape(-1,1)
dT_dt_SON_09 = np.mean(dT_dt_SON_09).reshape(-1,1)
dT_dt_SON_10 = np.mean(dT_dt_SON_10).reshape(-1,1)
dT_dt_SON_11 = np.mean(dT_dt_SON_11).reshape(-1,1)
dT_dt_SON_12 = np.mean(dT_dt_SON_12).reshape(-1,1)
dT_dt_SON_13 = np.mean(dT_dt_SON_13).reshape(-1,1)
dT_dt_SON_14 = np.mean(dT_dt_SON_14).reshape(-1,1)
dT_dt_SON_15 = np.mean(dT_dt_SON_15).reshape(-1,1)
dT_dt_SON_16 = np.mean(dT_dt_SON_16).reshape(-1,1)
dT_dt_SON_17 = np.mean(dT_dt_SON_17).reshape(-1,1)
dT_dt_SON_18 = np.mean(dT_dt_SON_18).reshape(-1,1)
dT_dt_SON_19 = np.mean(dT_dt_SON_19).reshape(-1,1)

In [60]:
R_recur_SON_93 = np.mean(R_recur_SON_93, axis = (1,2)).reshape(-1,1)
R_recur_SON_94 = np.mean(R_recur_SON_94, axis = (1,2)).reshape(-1,1)
R_recur_SON_95 = np.mean(R_recur_SON_95, axis = (1,2)).reshape(-1,1)
R_recur_SON_96 = np.mean(R_recur_SON_96, axis = (1,2)).reshape(-1,1)
R_recur_SON_97 = np.mean(R_recur_SON_97, axis = (1,2)).reshape(-1,1)
R_recur_SON_98 = np.mean(R_recur_SON_98, axis = (1,2)).reshape(-1,1)
R_recur_SON_99 = np.mean(R_recur_SON_99, axis = (1,2)).reshape(-1,1)
R_recur_SON_00 = np.mean(R_recur_SON_00, axis = (1,2)).reshape(-1,1)
R_recur_SON_01 = np.mean(R_recur_SON_01, axis = (1,2)).reshape(-1,1)
R_recur_SON_02 = np.mean(R_recur_SON_02, axis = (1,2)).reshape(-1,1)
R_recur_SON_03 = np.mean(R_recur_SON_03, axis = (1,2)).reshape(-1,1)
R_recur_SON_04 = np.mean(R_recur_SON_04, axis = (1,2)).reshape(-1,1)
R_recur_SON_05 = np.mean(R_recur_SON_05, axis = (1,2)).reshape(-1,1)
R_recur_SON_06 = np.mean(R_recur_SON_06, axis = (1,2)).reshape(-1,1)
R_recur_SON_07 = np.mean(R_recur_SON_07, axis = (1,2)).reshape(-1,1)
R_recur_SON_08 = np.mean(R_recur_SON_08, axis = (1,2)).reshape(-1,1)
R_recur_SON_09 = np.mean(R_recur_SON_09, axis = (1,2)).reshape(-1,1)
R_recur_SON_10 = np.mean(R_recur_SON_10, axis = (1,2)).reshape(-1,1)
R_recur_SON_11 = np.mean(R_recur_SON_11, axis = (1,2)).reshape(-1,1)
R_recur_SON_12 = np.mean(R_recur_SON_12, axis = (1,2)).reshape(-1,1)
R_recur_SON_13 = np.mean(R_recur_SON_13, axis = (1,2)).reshape(-1,1)
R_recur_SON_14 = np.mean(R_recur_SON_14, axis = (1,2)).reshape(-1,1)
R_recur_SON_15 = np.mean(R_recur_SON_15, axis = (1,2)).reshape(-1,1)
R_recur_SON_16 = np.mean(R_recur_SON_16, axis = (1,2)).reshape(-1,1)
R_recur_SON_17 = np.mean(R_recur_SON_17, axis = (1,2)).reshape(-1,1)
R_recur_SON_18 = np.mean(R_recur_SON_18, axis = (1,2)).reshape(-1,1)
R_recur_SON_19 = np.mean(R_recur_SON_19, axis = (1,2)).reshape(-1,1)

In [61]:
R_recur_SON_93 = np.mean(R_recur_SON_93).reshape(-1,1)
R_recur_SON_94 = np.mean(R_recur_SON_94).reshape(-1,1)
R_recur_SON_95 = np.mean(R_recur_SON_95).reshape(-1,1)
R_recur_SON_96 = np.mean(R_recur_SON_96).reshape(-1,1)
R_recur_SON_97 = np.mean(R_recur_SON_97).reshape(-1,1)
R_recur_SON_98 = np.mean(R_recur_SON_98).reshape(-1,1)
R_recur_SON_99 = np.mean(R_recur_SON_99).reshape(-1,1)
R_recur_SON_00 = np.mean(R_recur_SON_00).reshape(-1,1)
R_recur_SON_01 = np.mean(R_recur_SON_01).reshape(-1,1)
R_recur_SON_02 = np.mean(R_recur_SON_02).reshape(-1,1)
R_recur_SON_03 = np.mean(R_recur_SON_03).reshape(-1,1)
R_recur_SON_04 = np.mean(R_recur_SON_04).reshape(-1,1)
R_recur_SON_05 = np.mean(R_recur_SON_05).reshape(-1,1)
R_recur_SON_06 = np.mean(R_recur_SON_06).reshape(-1,1)
R_recur_SON_07 = np.mean(R_recur_SON_07).reshape(-1,1)
R_recur_SON_08 = np.mean(R_recur_SON_08).reshape(-1,1)
R_recur_SON_09 = np.mean(R_recur_SON_09).reshape(-1,1)
R_recur_SON_10 = np.mean(R_recur_SON_10).reshape(-1,1)
R_recur_SON_11 = np.mean(R_recur_SON_11).reshape(-1,1)
R_recur_SON_12 = np.mean(R_recur_SON_12).reshape(-1,1)
R_recur_SON_13 = np.mean(R_recur_SON_13).reshape(-1,1)
R_recur_SON_14 = np.mean(R_recur_SON_14).reshape(-1,1)
R_recur_SON_15 = np.mean(R_recur_SON_15).reshape(-1,1)
R_recur_SON_16 = np.mean(R_recur_SON_16).reshape(-1,1)
R_recur_SON_17 = np.mean(R_recur_SON_17).reshape(-1,1)
R_recur_SON_18 = np.mean(R_recur_SON_18).reshape(-1,1)
R_recur_SON_19 = np.mean(R_recur_SON_19).reshape(-1,1)

In [62]:
Q_net_SON_93_19 = np.concatenate((Q_net_SON_93, Q_net_SON_94, Q_net_SON_95, Q_net_SON_96,
                                 Q_net_SON_97, Q_net_SON_98, Q_net_SON_99, Q_net_SON_00, Q_net_SON_01,
                                 Q_net_SON_02, Q_net_SON_03, Q_net_SON_04, Q_net_SON_05, Q_net_SON_06,
                                 Q_net_SON_07, Q_net_SON_08, Q_net_SON_09, Q_net_SON_10, Q_net_SON_11,
                                 Q_net_SON_12, Q_net_SON_13, Q_net_SON_14, Q_net_SON_15, Q_net_SON_16,
                                 Q_net_SON_17, Q_net_SON_18, Q_net_SON_19), axis = 0)
Q_net_SON_93_19.shape

(27, 1)

(27, 1)

In [63]:
ZAdv_SON_93_19 = np.concatenate((ZAdv_SON_93, ZAdv_SON_94, ZAdv_SON_95, ZAdv_SON_96,
                                 ZAdv_SON_97, ZAdv_SON_98, ZAdv_SON_99, ZAdv_SON_00, ZAdv_SON_01,
                                 ZAdv_SON_02, ZAdv_SON_03, ZAdv_SON_04, ZAdv_SON_05, ZAdv_SON_06,
                                 ZAdv_SON_07, ZAdv_SON_08, ZAdv_SON_09, ZAdv_SON_10, ZAdv_SON_11,
                                 ZAdv_SON_12, ZAdv_SON_13, ZAdv_SON_14, ZAdv_SON_15, ZAdv_SON_16,
                                 ZAdv_SON_17, ZAdv_SON_18, ZAdv_SON_19), axis = 0)
ZAdv_SON_93_19.shape

(27, 1)

(27, 1)

In [64]:
MAdv_SON_93_19 = np.concatenate((MAdv_SON_93, MAdv_SON_94, MAdv_SON_95, MAdv_SON_96,
                                 MAdv_SON_97, MAdv_SON_98, MAdv_SON_99, MAdv_SON_00, MAdv_SON_01,
                                 MAdv_SON_02, MAdv_SON_03, MAdv_SON_04, MAdv_SON_05, MAdv_SON_06,
                                 MAdv_SON_07, MAdv_SON_08, MAdv_SON_09, MAdv_SON_10, MAdv_SON_11,
                                 MAdv_SON_12, MAdv_SON_13, MAdv_SON_14, MAdv_SON_15, MAdv_SON_16,
                                 MAdv_SON_17, MAdv_SON_18, MAdv_SON_19), axis = 0)
MAdv_SON_93_19.shape

(27, 1)

(27, 1)

In [65]:
VAdv_SON_93_19 = np.concatenate((VAdv_SON_93, VAdv_SON_94, VAdv_SON_95, VAdv_SON_96,
                                 VAdv_SON_97, VAdv_SON_98, VAdv_SON_99, VAdv_SON_00, VAdv_SON_01,
                                 VAdv_SON_02, VAdv_SON_03, VAdv_SON_04, VAdv_SON_05, VAdv_SON_06,
                                 VAdv_SON_07, VAdv_SON_08, VAdv_SON_09, VAdv_SON_10, VAdv_SON_11,
                                 VAdv_SON_12, VAdv_SON_13, VAdv_SON_14, VAdv_SON_15, VAdv_SON_16,
                                 VAdv_SON_17, VAdv_SON_18, VAdv_SON_19), axis = 0)
VAdv_SON_93_19.shape

(27, 1)

(27, 1)

In [66]:
R_origin_SON_93_19  = np.concatenate((R_origin_SON_93 , R_origin_SON_94, R_origin_SON_95, R_origin_SON_96,
                                 R_origin_SON_97, R_origin_SON_98, R_origin_SON_99, R_origin_SON_00, R_origin_SON_01,
                                 R_origin_SON_02, R_origin_SON_03, R_origin_SON_04, R_origin_SON_05, R_origin_SON_06,
                                 R_origin_SON_07, R_origin_SON_08, R_origin_SON_09, R_origin_SON_10, R_origin_SON_11,
                                 R_origin_SON_12, R_origin_SON_13, R_origin_SON_14, R_origin_SON_15, R_origin_SON_16,
                                 R_origin_SON_17, R_origin_SON_18, R_origin_SON_19), axis = 0)
R_origin_SON_93_19.shape

(27, 1)

(27, 1)

In [67]:
dT_dt_SON_93_19  = np.concatenate((dT_dt_SON_93 , dT_dt_SON_94, dT_dt_SON_95, dT_dt_SON_96,
                                 dT_dt_SON_97, dT_dt_SON_98, dT_dt_SON_99, dT_dt_SON_00, dT_dt_SON_01,
                                 dT_dt_SON_02, dT_dt_SON_03, dT_dt_SON_04, dT_dt_SON_05, dT_dt_SON_06,
                                 dT_dt_SON_07, dT_dt_SON_08, dT_dt_SON_09, dT_dt_SON_10, dT_dt_SON_11,
                                 dT_dt_SON_12, dT_dt_SON_13, dT_dt_SON_14, dT_dt_SON_15, dT_dt_SON_16,
                                 dT_dt_SON_17, dT_dt_SON_18, dT_dt_SON_19), axis = 0)
dT_dt_SON_93_19.shape

(27, 1)

(27, 1)

In [70]:
R_recur_SON_93_19  = np.concatenate((R_recur_SON_93 , R_recur_SON_94, R_recur_SON_95, R_recur_SON_96,
                                 R_recur_SON_97, R_recur_SON_98, R_recur_SON_99, R_recur_SON_00, R_recur_SON_01,
                                 R_recur_SON_02, R_recur_SON_03, R_recur_SON_04, R_recur_SON_05, R_recur_SON_06,
                                 R_recur_SON_07, R_recur_SON_08, R_recur_SON_09, R_recur_SON_10, R_recur_SON_11,
                                 R_recur_SON_12, R_recur_SON_13, R_recur_SON_14, R_recur_SON_15, R_recur_SON_16,
                                 R_recur_SON_17, R_recur_SON_18, R_recur_SON_19), axis = 0)
R_recur_SON_93_19.shape

(27, 1)

(27, 1)

In [69]:
np.savez(r'D:\heat_wave\WEIO\expand_WEIO\last\Figure1_MLD_eq\MLD_equa_SON.npz',
        Q_net_SON_93_19 = Q_net_SON_93_19, ZAdv_SON_93_19 = ZAdv_SON_93_19,
        MAdv_SON_93_19 = MAdv_SON_93_19, VAdv_SON_93_19 = VAdv_SON_93_19,
        R_origin_SON_93_19 = R_origin_SON_93_19, dT_dt_SON_93_19 = dT_dt_SON_93_19,
        R_recur_SON_93_19 = R_recur_SON_93_19)